# Creating Data Frames

#### Observed stocks:

In [ ]:
stock_list = ['DIS','T','NFLX','AMZN','BBY','CMG','KO','CL','COST','EQT','CVX','XOM','JPM','AXP','BAC','CVS','TECH','JNJ','BA','FDX','GE','AAPL','IBM','INTC','AMCR','ECL','NEM','IRM','AMT','EQIX','EXC','NRG','EIX']

#### Selected Sector's Stocks (Energy):

In [ ]:
energy_stocks = ['APA','BKR','CVX','COP','CTRA','DVN','FANG','EOG','EQT','XOM','HAL','HES','KMI','MRO','MPC','OXY','OKE','PSX','PXD','SLB','TRGP','VLO','WMB']

#### Creating json daily files:

In [ ]:
def get_jsonparsed_data(url):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

for i in stock_list:
    url = ("https://financialmodelingprep.com/api/v3/historical-price-full/{}?apikey=2f1c3c73de79230c2b4b038861ddf970".format(i))
    j = get_jsonparsed_data(url)
    df = pd.DataFrame(j['historical'])
    stock_name = j['symbol']
    df.to_csv('INSERT PATHWAY/df_daily/Observed/{}.csv'.format(stock_name), index=False)
    
for i in energy_stocks:
    url = ("https://financialmodelingprep.com/api/v3/historical-price-full/{}?apikey=2f1c3c73de79230c2b4b038861ddf970".format(i))
    j = get_jsonparsed_data(url)
    df = pd.DataFrame(j['historical'])
    stock_name = j['symbol']
    df.to_csv('INSERT PATHWAY/df_daily/Energy Sector/{}.csv'.format(stock_name), index=False)

#### Creating json hourly files:

In [ ]:
def get_jsonparsed_data(url):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

for i in stock_list:
    url = ("https://financialmodelingprep.com/api/v3/historical-chart/1hour/{}?apikey=2f1c3c73de79230c2b4b038861ddf970".format(i))
    j = get_jsonparsed_data(url)
    df = pd.DataFrame(j)
    stock_name = i
    df.to_csv('INSERT PATHWAY/df_hourly/Observed/{}.csv'.format(stock_name), index=False)
    
for i in energy_stocks:
    url = ("https://financialmodelingprep.com/api/v3/historical-price-full/{}?apikey=2f1c3c73de79230c2b4b038861ddf970".format(i))
    j = get_jsonparsed_data(url)
    df = pd.DataFrame(j['historical'])
    stock_name = j['symbol']
    df.to_csv('INSERT PATHWAY/df_hourly/Energy Sector/{}.csv'.format(stock_name), index=False)